need to install google-cloud-vision (GCP SDK)

conda create -n gcp-cloud  
conda install -c conda-forge pillow=10.1.0 pandas=2.1.2 google-cloud-vision=3.4.5 scikit-learn=1.3.2 ipykernel jupyterlab notebook python=3.12.0  


how to set up env as new kernel in jupyterlabs:  

python -m ipykernel install --user --name=gcp-cloud

In [1]:
# Authentication to Google API
import os
import io
import math
from numpy import random
from collections import Counter
from google.cloud import vision
from my_timer import my_timer
from Pillow_Library import draw_borders
from PIL import Image
import re
import pandas as pd     

In [2]:
# setup the location to our api key credential file (see previous video for details)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='vision_key.json'

In [3]:
# Create Google Vision API client
vision_client = vision.ImageAnnotatorClient()

In [4]:
# select the file we will be processing
image_path = 'bicycle.jpg'
# file = 'original.jpg'
# image_path = os.path.join('.\\content', file)

In [5]:
# read the file into a byte array using the IO python package
with io.open(image_path, 'rb') as image_file:
    image_content = image_file.read()
type(image_content)   

bytes

In [6]:
# create Google Cloud Vision Image from the byte array we created for our photo earlier.
image = vision.Image(content=image_content)       

In [7]:
# pass in the image to the object method and get a response back from the GCP service
response = vision_client.object_localization(image=image) 
print(response)              

localized_object_annotations {
  mid: "/m/0199g"
  name: "Bicycle"
  score: 0.7729883193969727
  bounding_poly {
    normalized_vertices {
      x: 0.46875
      y: 0.4296875
    }
    normalized_vertices {
      x: 0.84375
      y: 0.4296875
    }
    normalized_vertices {
      x: 0.84375
      y: 0.98828125
    }
    normalized_vertices {
      x: 0.46875
      y: 0.98828125
    }
  }
}
localized_object_annotations {
  mid: "/m/0h9mv"
  name: "Tire"
  score: 0.7600739002227783
  bounding_poly {
    normalized_vertices {
      x: 0.71875
      y: 0.59765625
    }
    normalized_vertices {
      x: 0.84765625
      y: 0.59765625
    }
    normalized_vertices {
      x: 0.84765625
      y: 0.92578125
    }
    normalized_vertices {
      x: 0.71875
      y: 0.92578125
    }
  }
}
localized_object_annotations {
  mid: "/m/0h9mv"
  name: "Tire"
  score: 0.7524343132972717
  bounding_poly {
    normalized_vertices {
      x: 0.46875
      y: 0.609375
    }
    normalized_vertices {
      

In [8]:
# extract the annotations from within the response
localized_object_annotations = response.localized_object_annotations
print(localized_object_annotations)   

[mid: "/m/0199g"
name: "Bicycle"
score: 0.7729883193969727
bounding_poly {
  normalized_vertices {
    x: 0.46875
    y: 0.4296875
  }
  normalized_vertices {
    x: 0.84375
    y: 0.4296875
  }
  normalized_vertices {
    x: 0.84375
    y: 0.98828125
  }
  normalized_vertices {
    x: 0.46875
    y: 0.98828125
  }
}
, mid: "/m/0h9mv"
name: "Tire"
score: 0.7600739002227783
bounding_poly {
  normalized_vertices {
    x: 0.71875
    y: 0.59765625
  }
  normalized_vertices {
    x: 0.84765625
    y: 0.59765625
  }
  normalized_vertices {
    x: 0.84765625
    y: 0.92578125
  }
  normalized_vertices {
    x: 0.71875
    y: 0.92578125
  }
}
, mid: "/m/0h9mv"
name: "Tire"
score: 0.7524343132972717
bounding_poly {
  normalized_vertices {
    x: 0.46875
    y: 0.609375
  }
  normalized_vertices {
    x: 0.640625
    y: 0.609375
  }
  normalized_vertices {
    x: 0.640625
    y: 0.9765625
  }
  normalized_vertices {
    x: 0.46875
    y: 0.9765625
  }
}
, mid: "/m/01xygc"
name: "Coat"
score: 0.

In [9]:
# open the original image using the PIL library
pillow_image = Image.open(image_path)

In [10]:
# create a list of objects found
df_list = []
for obj in localized_object_annotations:
    # append each item to that list
    df_list.append({'object': obj.name, 'score': obj.score}) 
    
    # pick a different color for each border
    import random

    r, g, b = random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)


    # add text and borders to the original image
    draw_borders(pillow_image, obj.bounding_poly, (r, g, b),
                pillow_image.size, obj.name, obj.score) 

In [11]:
# convert python list to pandas dataframe
df = pd.DataFrame(columns=['object', 'score'])
df = pd.DataFrame.from_records(df_list) 
print(df)

          object     score
0        Bicycle  0.772988
1           Tire  0.760074
2           Tire  0.752434
3           Coat  0.747894
4  Bicycle wheel  0.704720
5  Bicycle wheel  0.691584
6      Outerwear  0.689558
7          Pants  0.652979
8          Wheel  0.647801
9         Person  0.642258


In [12]:
# our image with text and borders   
pillow_image.show() 